# OCR Projet 6 - Anticipez les besoins en consommation de bâtiments

# Partie 2 : Modélisation 

## 1.  Import des modules 

In [341]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import missingno as msno
from sklearn.preprocessing import StandardScaler, FunctionTransformer


print(f"Pandas      : {pd.__version__}")
print(f"Numpy       : {np.__version__}")
print(f"Seaborn     : {sns.__version__}")
print(f"Plotly      : {plotly.__version__}")
print(f"Missingno   : {msno.__version__}")
print(f"StandardScaler : {StandardScaler}")

Pandas      : 2.3.3
Numpy       : 2.3.4
Seaborn     : 0.13.2
Plotly      : 6.4.0
Missingno   : 0.5.2
StandardScaler : <class 'sklearn.preprocessing._data.StandardScaler'>


In [342]:
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from category_encoders.binary import BinaryEncoder

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


## 2. Preprocessing

A déjà été réalisé dans l'analyse exploration : 
* Suppression de toutes les colonnes peu pertinentes pour la modélisation. 
* Tracer la distribution de la cible
* Suppression des outliers (Z) 
* Suppression des features redondantes en utilisant une matrice de corrélation
* Réalisation différents graphiques pour comprendre le lien entre vos features et la target (boxplots, scatterplots)

### 2.1 Séparation du  jeu de données features et targets 

In [343]:
df_building_clean = pd.read_csv('data/building_clean.csv')

In [344]:
df_building_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331 entries, 0 to 1330
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PrimaryPropertyType     1331 non-null   object 
 1   Latitude                1331 non-null   float64
 2   Longitude               1331 non-null   float64
 3   YearBuilt               1331 non-null   int64  
 4   NumberofFloors          1331 non-null   int64  
 5   PropertyGFAParking      1331 non-null   int64  
 6   PropertyGFABuilding(s)  1331 non-null   int64  
 7   SiteEnergyUse(kBtu)     1331 non-null   float64
 8   TotalGHGEmissions       1331 non-null   float64
 9   pct_electricity         1331 non-null   float64
 10  pct_steam               1331 non-null   float64
 11  has_parking             1331 non-null   bool   
dtypes: bool(1), float64(6), int64(4), object(1)
memory usage: 115.8+ KB


 targets : consommation des bâtiments

In [345]:
y = df_building_clean['SiteEnergyUse(kBtu)']

In [346]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1331 entries, 0 to 1330
Series name: SiteEnergyUse(kBtu)
Non-Null Count  Dtype  
--------------  -----  
1331 non-null   float64
dtypes: float64(1)
memory usage: 10.5 KB


 features

In [350]:
features = df_building_clean.drop(columns=['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']).columns.tolist()
features

['PrimaryPropertyType',
 'Latitude',
 'Longitude',
 'YearBuilt',
 'NumberofFloors',
 'PropertyGFAParking',
 'PropertyGFABuilding(s)',
 'pct_electricity',
 'pct_steam',
 'has_parking']

In [351]:
X = df_building_clean[features]

### 2.2 Encodage variable catégorielle

In [352]:
df_building_clean['PrimaryPropertyType'].value_counts()

PrimaryPropertyType
Office              408
Other               350
Warehouse           234
Retail              120
Education            84
Worship Facility     66
Hospitality          54
Healthcare           15
Name: count, dtype: int64

J'ai 8 catégories dans PrimaryPropertyType.

*Choix méthode d'encodage* : Pour encoder la variable catégorielle, LabelEncoder met un nombre à chaque catégorie, ce qui crée une hiérarchie implicite, alors que One-Hot crée une colonne pour chaque catégorie sans ordre.
On choisit One-Hot parce que nos types de bâtiments n’ont pas d’ordre, donc on évite de tromper le modèle.

In [353]:
# Utilisation de pd.get_dummies
X = pd.get_dummies(X, columns=['PrimaryPropertyType'], drop_first=True) 

# drop_first=True pour éviter la colinéarité
print(X.head())

   Latitude  Longitude  YearBuilt  NumberofFloors  PropertyGFAParking  \
0  47.61220 -122.33799       1927              12                   0   
1  47.61317 -122.33393       1996              11               15064   
2  47.61412 -122.33664       1926              10                   0   
3  47.61623 -122.33657       1999               2               37198   
4  47.61390 -122.33283       1926              11                   0   

   PropertyGFABuilding(s)  pct_electricity  pct_steam  has_parking  \
0                   88434         0.546060   0.277302        False   
1                   88502         0.386609   0.000000         True   
2                   61320         0.407519   0.325913        False   
3                   60090         0.609884   0.000000         True   
4                   83008         0.488160   0.000000        False   

   PrimaryPropertyType_Healthcare  PrimaryPropertyType_Hospitality  \
0                           False                             True   


One-Hot crée une colonne par catégorie (-1 pour la derniere colonne) On a donc 7 colonnes de plus. C'est raisonnable car on a un petit panel mais on peut essayer l'**encodage binaire** avec la librairy category_encoders.

#### Test binaryEncoder

In [354]:


encoder = BinaryEncoder(cols=['PrimaryPropertyType'], return_df=True)
X_binary_encoded = encoder.fit_transform(df_building_clean)
X_binary_encoded.head()

,PrimaryPropertyType_0,PrimaryPropertyType_1,PrimaryPropertyType_2,PrimaryPropertyType_3,Latitude,Longitude,YearBuilt,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),SiteEnergyUse(kBtu),TotalGHGEmissions,pct_electricity,pct_steam,has_parking
0,0,0,0,1,47.61220,-122.33799,1927,12,0,88434,7226362.5,249.98,0.546060,0.277302,False
1,0,0,0,1,47.61317,-122.33393,1996,11,15064,88502,8387933.0,295.86,0.386609,0.000000,True
2,0,0,0,1,47.61412,-122.33664,1926,10,0,61320,6794584.0,286.43,0.407519,0.325913,False
3,0,0,1,0,47.61623,-122.33657,1999,2,37198,60090,12086616.0,301.81,0.609884,0.000000,True
4,0,0,0,1,47.61390,-122.33283,1926,11,0,83008,5758795.0,176.14,0.488160,0.000000,False


Cette méthode d’encodage est moins lisible, donc je préfère rester sur la première méthode, qui est plus claire et suffisante vu la petite taille de mes données.

### 2.3 Train-test split

Train test split 

In [355]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

In [356]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1064 entries, 584 to 1126
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Latitude                              1064 non-null   float64
 1   Longitude                             1064 non-null   float64
 2   YearBuilt                             1064 non-null   int64  
 3   NumberofFloors                        1064 non-null   int64  
 4   PropertyGFAParking                    1064 non-null   int64  
 5   PropertyGFABuilding(s)                1064 non-null   int64  
 6   pct_electricity                       1064 non-null   float64
 7   pct_steam                             1064 non-null   float64
 8   has_parking                           1064 non-null   bool   
 9   PrimaryPropertyType_Healthcare        1064 non-null   bool   
 10  PrimaryPropertyType_Hospitality       1064 non-null   bool   
 11  PrimaryPropertyType_

### 2.4 Normalisation : mettre à la même échelle 

La normalisation rend chaque variable comparable (moyenne = 0, écart-type = 1).

In [357]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform (X_test)

## 3. Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

Mon objectif est de prédire les consommations d'énergies : Expliquer/prédire une variable cible. La régression linéaire réponds à cet objectif

### 3.1 fonctions 

In [358]:
def evaluate(y_test, y_pred, y_train, y_pred_train):

  # Calcul des métriques :
  mae_train = mean_absolute_error(y_train, y_pred_train)
  mae_test = mean_absolute_error(y_test, y_pred)

  mse_train = mean_squared_error(y_train, y_pred_train)
  mse_test = mean_squared_error(y_test, y_pred)

  rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
  rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

  r2_train = r2_score(y_train, y_pred_train)
  r2_test = r2_score(y_test, y_pred)

    # Affichage :
  print(f"MAE train: {mae_train:.0f} \t\t MAE test: {mae_test:.0f}")
  # print(f"MSE train: {mse_train:.0f} \t\t MSE test: {mse_test:.0f}")
  print(f"RMSE train: {rmse_train:.2f} \t\t RMSE test: {rmse_test:.2f}")
  print(f"R2 train: {r2_train:.3f} \t\t R2 test: {r2_test:.3f}\n")

  # MAE - Erreur Absolue Moyenne
  print(f"L'erreur absolue moyenne (MAE) est de {mae_train:.0f} unités (kBtu) sur le train et {mae_test:.0f} unités sur le test.")
  print(f"→ Le modèle se trompe en moyenne de {mae_train:.0f} unités (kBtu) sur le train et de {mae_test:.0f} unités sur le test.")
  print()

  # # MSE - Erreur Quadratique Moyenne
  # print(f"→ Les erreurs au carré (MSE) valent en moyenne {mse_train:.2f} sur le train et {mse_test:.2f} sur le test (pénalise les grosses erreurs).")
  # print()

  # RMSE - Racine de l'Erreur Quadratique Moyenne
  print(f"RMSE : L'écart-type des erreurs est de {rmse_train:.2f} unités sur le train et {rmse_test:.2f} unités sur le test.")
  print()

  # R² - Coefficient de détermination
  print(f"R² : Le coefficient de détermination est de {r2_train:.3f} sur le train et {r2_test:.3f} sur le test.")
  print(f"→ Le modèle explique {r2_train*100:.1f}% de la variance sur le train et {r2_test*100:.1f}% sur le test.")
  print()

  print('_' * 50)

### 3.2 Regression linéaire

In [359]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [360]:
#R²
lin_reg.score(X_test, y_test)

0.5185017151088158

Cela signifit que le modèle explique environ 52 % de la variance de la variable cible — donc il capte une partie du signal, mais il y a encore presque la moitié qui lui échappe.

je prédit y en fonction des données X_train 

In [361]:
y_train_pred = lin_reg.predict(X_train)

In [362]:
y_pred_train = lin_reg.predict(X_train)

In [ ]:
-> regarder scoreF1

In [363]:
evaluate(y_test, y_pred_test, y_train, y_pred_train)

MAE train: 1617931 		 MAE test: 1932012
RMSE train: 2354920.48 		 RMSE test: 2932246.19
R2 train: 0.551 		 R2 test: 0.520

L'erreur absolue moyenne (MAE) est de 1617931 unités (kBtu) sur le train et 1932012 unités sur le test.
→ Le modèle se trompe en moyenne de 1617931 unités (kBtu) sur le train et de 1932012 unités sur le test.

RMSE : L'écart-type des erreurs est de 2354920.48 unités sur le train et 2932246.19 unités sur le test.

R² : Le coefficient de détermination est de 0.551 sur le train et 0.520 sur le test.
→ Le modèle explique 55.1% de la variance sur le train et 52.0% sur le test.

__________________________________________________


In [364]:
y.describe()

count    1.331000e+03
mean     3.614384e+06
std      3.671904e+06
min      5.713320e+04
25%      1.124243e+06
50%      2.171752e+06
75%      4.936747e+06
max      2.239442e+07
Name: SiteEnergyUse(kBtu), dtype: float64

faut il normaliser y ? 

### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn. 

random forest , regression logistic ? lequel s'adapte au contexte.

In [ ]:
# CODE OPTIMISATION ET INTERPRETATION DU MODELE